# Transformer

## 導入

Transformerは、損失が下がる理由を『式』と『数値変化』の両方で確認する構成です。

前提: ベクトル・行列の計算と、微分の意味を言葉で説明できると理解しやすくなります。

到達目標: 損失、勾配、最適化、正則化のつながりをコードで確かめ、モデルの挙動を説明できる状態にします。

ここで扱う中心語は 「順伝播」、「逆伝播」、「勾配」、「損失」、「正則化」、「Transformer」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは ディープラーニング 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 観察 1: 順伝播の最小例

まず、1ユニットの順伝播を自分で計算します。ライブラリを使わない形で内部計算を見える化するのが目的です。

In [ ]:
import math
x = [0.8, -0.4, 0.2]
w = [0.3, -0.6, 0.5]
z = sum(xi * wi for xi, wi in zip(x, w)) + 0.1
y = 1 / (1 + math.exp(-z))
print('z=', round(z, 4), 'sigmoid=', round(y, 4))

ここで `z` は線形変換、`y` は非線形変換の結果です。この二段がニューラルネットワークの最小単位になります。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 観察 2: 損失を計算する

次に、予測値に対する損失を計算します。学習は損失を下げる方向に進むので、損失の意味を言葉で説明できることが重要です。

In [ ]:
target = 1.0
eps = 1e-9
loss = -(target * math.log(y + eps) + (1 - target) * math.log(1 - y + eps))
print('loss =', round(loss, 6))
print('error =', round(target - y, 6))

損失は『どれだけ外したか』を測る物差しです。物差しがない状態では、改善の方向を決められません。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 計算の対応表

1. Attention(Q,K,V) = softmax(QK^T / sqrt(d_k))V
2. FFN(x) = W2 * sigma(W1*x + b1) + b2

## 観察 3: 勾配降下法の一歩

ここで、重みを一回更新する最小実験を行います。更新前後の損失を比較して、学習の方向が正しいかを確認します。

In [ ]:
lr = 0.1
grad = -(target - y) * y * (1 - y)
w_new = [wi - lr * grad * xi for wi, xi in zip(w, x)]
z_new = sum(xi * wi for xi, wi in zip(x, w_new)) + 0.1
y_new = 1 / (1 + math.exp(-z_new))
print('y before/after =', round(y, 6), round(y_new, 6))

この更新で損失が下がれば、勾配方向が合理的だったと言えます。下がらないなら学習率や符号を疑います。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 観察 4: 正則化の感覚を作る

次に、重みが大きくなりすぎることを抑える正則化項を追加します。過学習対策をコードで体験するのが狙いです。

In [ ]:
l2 = 0.01
weight_norm = sum(wi * wi for wi in w_new)
loss_reg = loss + l2 * weight_norm
print('weight_norm =', round(weight_norm, 6))
print('regularized loss =', round(loss_reg, 6))

正則化は精度を上げる魔法ではなく、汎化の崩れを防ぐ保険です。評価データと合わせて効果を判断してください。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 観察 5: ミニバッチの考え方

最後に、複数サンプルをまとめて扱う発想を確認します。実務では1件ずつよりバッチ処理が主流です。

In [ ]:
batch = [[0.8, -0.4, 0.2], [0.2, 0.1, -0.3], [0.5, -0.2, 0.7]]
targets = [1.0, 0.0, 1.0]
preds = []
for bx in batch:
    z_b = sum(xi * wi for xi, wi in zip(bx, w_new)) + 0.1
    preds.append(1 / (1 + math.exp(-z_b)))
print('preds =', [round(p, 4) for p in preds])

ミニバッチを扱えるようになると、計算効率と学習安定性の両面で設計の幅が広がります。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 要点整理

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 学習率が大きすぎて発散する
2. 検証損失の監視をせず過学習を見逃す
3. 前処理と活性化の相性を無視する

次は「自然言語処理」へ進み、今回のコードと何が変わるかを比較してください。